## miRNA for datafreeze

This notebook combines the miRNA data from the WashU pipeline. The format of the output file is for the datafreeze according to Yize's instructions: rows are miRNA identifiers and columns are Patient_IDs. Normal samples have a ".A" appended to the Patient_ID and tumor samples have a ".T". Replicates have a ".1" appended to the Patient_ID.

In [70]:
import pandas as pd
import numpy as np
import warnings
from functools import reduce
import re
import glob

In [2]:
''' Adds '_i' to duplicate col names where the int i increases by one with each duplicate 
Returns a df with unique column names. '''

def rename_duplicate_index(df):
    cols = pd.Series(df.index[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.index=cols
    return df

In [3]:
cancer_names = ['GBM','HNSCC','LSCC','LUAD','EC','PDA','ccRCC']
identifier_list = ['total','precursor', 'mature']
#identifier_list = ['total']

# Make all combined files

In [ ]:
# Create all files for yize
for identifier in identifier_list:
    print('making '+identifier+' files')
    for ca in cancer_names:
        print(ca)
        path = '../../../miRNA/'+ca +'/WashU_pipeline/' 
        all_files = glob.glob(path + '*'+identifier+'.txt.gz') 
        
        all_df = pd.DataFrame()

        for filename in all_files:
            if ca == 'LUAD':
                if re.search('\w3\w-\d\d\d\d\d\.[AT]', filename) != None:
                    patient = re.search('\w3\w-\d\d\d\d\d\.[AT]', filename).group()
                else: 
                    patient = re.search('\d\dLU\d\d\d\.[AT]', filename).group()
            else:
                patient = re.search('\w3\w-\d\d\d\d\d\.[AT]', filename).group()
            
            # Use different cols in files
            if identifier == 'precursor':
                df = pd.read_csv(filename, sep = '\t', index_col=['Name', 'ID', 'Alias'])
                df = df.drop(columns = ['primary_miR_Count_Raw']) 
            elif identifier == 'mature':
                df = pd.read_csv(filename, sep = '\t', index_col=['Name', 'ID', 'Alias', 'Derives_from'])
                df = df.drop(columns = ['miR_Count_Raw'])
            else:
                df = pd.read_csv(filename, sep = '\t', index_col=['Name', 'ID', 'Alias'])
                df = df.drop(columns = ['Count_Raw'])
                
            df = df.transpose()
            df['Patient_ID'] = patient
            df = df.set_index('Patient_ID')
            all_df = all_df.append(df) # change to join?
        
        # Create unique identifiers for duplicate Patient_IDs
        all_df = rename_duplicate_index(all_df)

        # sort values
        normal = all_df.loc[all_df.index.str.contains('.A$')]
        normal = normal.sort_values(by=["Patient_ID"])
        tumor = all_df.loc[~ all_df.index.str.contains('.A$')] # doesn't contain A (includes .T.1 for replicates)
        tumor = tumor.sort_values(by=["Patient_ID"])
        all_df = tumor.append(normal)

        # Create files for datafreeze (identifier cols, miRNA rows)
        transposed_df = all_df.transpose()
        transposed_df.to_csv('~/Documents/miRNA/datafreeze/'+ca+'_'+identifier+'_miRNA_combined.tsv', sep = '\t', na_rep = 'NA') #yize

making total files
GBM
HNSCC
LSCC
LUAD
EC
PDA
ccRCC
making precursor files
GBM
HNSCC
LSCC
LUAD
EC
PDA
ccRCC
making mature files
GBM
HNSCC
LSCC


# Test individually

In [4]:
cancer_names = ['GBM','HNSCC','LSCC','LUAD','EC','PDA','ccRCC']
identifier_list = ['total','precursor', 'mature']

In [64]:
ca = cancer_names[6]
identifier = identifier_list[0] 

path = '../../../miRNA/'+ca +'/WashU_pipeline/' 
all_files = glob.glob(path + '*'+identifier+'.txt.gz') 
print(ca)
print(identifier)
print('Num files:', len(all_files))

ccRCC
total
Num files: 185


In [29]:
all_df = pd.DataFrame()

for filename in all_files:
    if ca == 'LUAD':
        if re.search('\w3\w-\d\d\d\d\d\.[AT]', filename) != None:
            patient = re.search('\w3\w-\d\d\d\d\d\.[AT]', filename).group()
        else: 
            patient = re.search('\d\dLU\d\d\d\.[AT]', filename).group()
    else:
        patient = re.search('\w3\w-\d\d\d\d\d\.[AT]', filename).group()

    if identifier == 'precursor':
        df = pd.read_csv(filename, sep = '\t', index_col=['Name', 'ID', 'Alias'])
        df = df.drop(columns = ['primary_miR_Count_Raw']) 
    elif identifier == 'mature':
        df = pd.read_csv(filename, sep = '\t', index_col=['Name', 'ID', 'Alias', 'Derives_from'])
        df = df.drop(columns = ['miR_Count_Raw'])
    else:
        df = pd.read_csv(filename, sep = '\t', index_col=['Name', 'ID', 'Alias'])
        df = df.drop(columns = ['Count_Raw'])

    df = df.transpose()
    df['Patient_ID'] = patient
    df = df.set_index('Patient_ID')
    all_df = all_df.append(df) # change to join?

# Create unique identifiers for duplicate Patient_IDs
all_df = rename_duplicate_index(all_df)

# sort values
normal = all_df.loc[all_df.index.str.contains('.A$')]
normal = normal.sort_values(by=["Patient_ID"])
tumor = all_df.loc[~ all_df.index.str.contains('.A$')] # doesn't contain A (includes .T.1 for replicates)
tumor = tumor.sort_values(by=["Patient_ID"])
all_df = tumor.append(normal)

# Create files for datafreeze (identifier cols, miRNA rows)
all_df = all_df.transpose()
#all_df.to_csv('~/Documents/miRNA/datafreeze/'+ca+'_'+identifier+'_miRNA_combined.tsv', sep = '\t', na_rep = 'NA') #yize

In [31]:
len(all_df.columns)

99

# Test created files

In [4]:
cancer_names = ['GBM','HNSCC','LSCC','LUAD','EC','PDA','ccRCC']
identifier_list = ['total','precursor', 'mature']

In [69]:
ca = cancer_names[6]
identifier = identifier_list[2] 

path = '../../../miRNA/datafreeze/'
fn = ca+'_'+identifier+'_miRNA_combined.tsv'

In [68]:
if identifier == 'mature':
    all_df = pd.read_csv(path+fn, delimiter = '\t', index_col = ['Name', 'ID','Alias', 'Derives_from'])
else:
    all_df = pd.read_csv(path+fn, delimiter = '\t', index_col = ['Name', 'ID','Alias'])
all_df = all_df.T

print(ca, identifier)

# check tumor/normal counts
tumor = all_df.loc[~ all_df.index.str.contains('.A$')]
print('Tumor:', len(tumor))
normal = all_df.loc[all_df.index.str.contains('.A$')]
print('Normal:', len(normal))

# check for duplicate indices 
if True in (all_df.index.duplicated()):
    print('Duplicates present')
else:
    print('NO dup index')

all_df

ccRCC mature
Tumor: 110
Normal: 75
NO dup index


Name         hsa-let-7a-2-3p hsa-let-7a-3p                hsa-let-7a-5p  \
ID              MIMAT0010195  MIMAT0004481 MIMAT0004481_1  MIMAT0000062   
Alias           MIMAT0010195  MIMAT0004481   MIMAT0004481  MIMAT0000062   
Derives_from       MI0000061     MI0000062      MI0000060     MI0000061   
C3L-00004.T              NaN     57.081886      48.300058  33300.694204   
C3L-00010.T              NaN     41.253197      75.005813  46274.836300   
C3L-00011.T              NaN     22.135035      38.736311  38481.757964   
C3L-00026.T              NaN     43.049636      32.287227  30000.215248   
C3L-00079.T              NaN     57.970268      96.617113  42144.384613   
...                      ...           ...            ...           ...   
C3N-01646.A              NaN     90.432266      83.475938  32757.349361   
C3N-01648.A              NaN     84.388186     111.238972  30690.448792   
C3N-01649.A              NaN     59.224665      71.915664  33555.002792   
C3N-01651.A              NaN     93.273638      76.617631  35097.537576   
C3N-01808.A              NaN     87.405372     111.684641  36564.580430   

Name                                       hsa-let-7b-3p hsa-let-7b-5p  \
ID           MIMAT0000062_1 MIMAT0000062_2  MIMAT0004482  MIMAT0000063   
Alias          MIMAT0000062   MIMAT0000062  MIMAT0004482  MIMAT0000063   
Derives_from      MI0000062      MI0000060     MI0000063     MI0000063   
C3L-00004.T    32962.593801   33630.012778    127.336515  12610.705927   
C3L-00010.T    46473.601704   45971.062757    138.760754  17836.382320   
C3L-00011.T    38005.854717   37568.687780    420.565661  58525.031957   
C3L-00026.T    30511.429678   29881.828749     91.480477  10783.933876   
C3L-00079.T    43033.262051   44224.873110    161.028521  14924.123361   
...                     ...            ...           ...           ...   
C3N-01646.A    33320.811943   33310.377450    139.126563  14114.389860   
C3N-01648.A    30644.418872   30767.165324     99.731492  12159.570387   
C3N-01649.A    34659.119752   33774.980117    287.662656  15051.525458   
C3N-01651.A    35377.358491   35450.644921    193.209679  13211.544611   
C3N-01808.A    36826.796545   36812.228983    233.080991  17471.362601   

Name         hsa-let-7c-3p hsa-let-7c-5p  ... hsa-miR-9851-5p hsa-miR-9898  \
ID            MIMAT0026472  MIMAT0000064  ...    MIMAT0048639 MIMAT0039318   
Alias         MIMAT0026472  MIMAT0000064  ...    MIMAT0048639 MIMAT0039318   
Derives_from     MI0000064     MI0000064  ...       MI0039502    MI0031826   
C3L-00004.T            NaN   6862.999082  ...             NaN          NaN   
C3L-00010.T            NaN   8441.904248  ...             NaN          NaN   
C3L-00011.T            NaN   7155.149993  ...             NaN          NaN   
C3L-00026.T            NaN   5811.700891  ...             NaN     5.381205   
C3L-00079.T            NaN   5513.616572  ...             NaN          NaN   
...                    ...           ...  ...             ...          ...   
C3N-01646.A      10.434492   5161.595503  ...             NaN          NaN   
C3N-01648.A      11.507480   4188.722670  ...             NaN          NaN   
C3N-01649.A       4.230333   5283.686143  ...             NaN     4.230333   
C3N-01651.A       3.331201   4563.745869  ...             NaN     3.331201   
C3N-01808.A            NaN   5618.223049  ...             NaN          NaN   

Name         hsa-miR-9899 hsa-miR-9901 hsa-miR-9903 hsa-miR-9985  \
ID           MIMAT0039319 MIMAT0039321 MIMAT0039323 MIMAT0039763   
Alias        MIMAT0039319 MIMAT0039321 MIMAT0039323 MIMAT0039763   
Derives_from    MI0031827    MI0031829    MI0031831    MI0032313   
C3L-00004.T           NaN          NaN          NaN    30.736400   
C3L-00010.T           NaN          NaN          NaN    33.752616   
C3L-00011.T           NaN    11.067517          NaN          NaN   
C3L-00026.T           NaN    16.143614          NaN    37.668432   
C3L-00079.T           NaN          NaN          

In [61]:
test = '\.1'
all_df.loc[all_df.index.str.contains(test)]

Empty DataFrame
Columns: [(hsa-let-7a-2-3p, MIMAT0010195, MIMAT0010195, MI0000061), (hsa-let-7a-3p, MIMAT0004481, MIMAT0004481, MI0000062), (hsa-let-7a-3p, MIMAT0004481_1, MIMAT0004481, MI0000060), (hsa-let-7a-5p, MIMAT0000062, MIMAT0000062, MI0000061), (hsa-let-7a-5p, MIMAT0000062_1, MIMAT0000062, MI0000062), (hsa-let-7a-5p, MIMAT0000062_2, MIMAT0000062, MI0000060), (hsa-let-7b-3p, MIMAT0004482, MIMAT0004482, MI0000063), (hsa-let-7b-5p, MIMAT0000063, MIMAT0000063, MI0000063), (hsa-let-7c-3p, MIMAT0026472, MIMAT0026472, MI0000064), (hsa-let-7c-5p, MIMAT0000064, MIMAT0000064, MI0000064), (hsa-let-7d-3p, MIMAT0004484, MIMAT0004484, MI0000065), (hsa-let-7d-5p, MIMAT0000065, MIMAT0000065, MI0000065), (hsa-let-7e-3p, MIMAT0004485, MIMAT0004485, MI0000066), (hsa-let-7e-5p, MIMAT0000066, MIMAT0000066, MI0000066), (hsa-let-7f-1-3p, MIMAT0004486, MIMAT0004486, MI0000067), (hsa-let-7f-2-3p, MIMAT0004487, MIMAT0004487, MI0000068), (hsa-let-7f-5p, MIMAT0000067, MIMAT0000067, MI0000067), (hsa-let-7f-5p, MIMAT0000067_1, MIMAT0000067, MI0000068), (hsa-let-7g-3p, MIMAT0004584, MIMAT0004584, MI0000433), (hsa-let-7g-5p, MIMAT0000414, MIMAT0000414, MI0000433), (hsa-let-7i-3p, MIMAT0004585, MIMAT0004585, MI0000434), (hsa-let-7i-5p, MIMAT0000415, MIMAT0000415, MI0000434), (hsa-miR-1-3p, MIMAT0000416, MIMAT0000416, MI0000437), (hsa-miR-1-3p, MIMAT0000416_1, MIMAT0000416, MI0000651), (hsa-miR-1-5p, MIMAT0031892, MIMAT0031892, MI0000651), (hsa-miR-100-3p, MIMAT0004512, MIMAT0004512, MI0000102), (hsa-miR-100-5p, MIMAT0000098, MIMAT0000098, MI0000102), (hsa-miR-101-2-5p, MIMAT0037312, MIMAT0037312, MI0000739), (hsa-miR-101-3p, MIMAT0000099, MIMAT0000099, MI0000103), (hsa-miR-101-3p, MIMAT0000099_1, MIMAT0000099, MI0000739), (hsa-miR-101-5p, MIMAT0004513, MIMAT0004513, MI0000103), (hsa-miR-10226, MIMAT0041128, MIMAT0041128, MI0033056), (hsa-miR-10392-3p, MIMAT0041616, MIMAT0041616, MI0033416), (hsa-miR-10392-5p, MIMAT0041615, MIMAT0041615, MI0033416), (hsa-miR-10393-3p, MIMAT0041618, MIMAT0041618, MI0033417), (hsa-miR-10393-5p, MIMAT0041617, MIMAT0041617, MI0033417), (hsa-miR-10394-3p, MIMAT0041620, MIMAT0041620, MI0033418), (hsa-miR-10394-5p, MIMAT0041619, MIMAT0041619, MI0033418), (hsa-miR-10395-3p, MIMAT0041622, MIMAT0041622, MI0033419), (hsa-miR-10395-5p, MIMAT0041621, MIMAT0041621, MI0033419), (hsa-miR-10396a-3p, MIMAT0041624, MIMAT0041624, MI0033420), (hsa-miR-10396a-5p, MIMAT0041623, MIMAT0041623, MI0033420), (hsa-miR-10396b-3p, MIMAT0041636, MIMAT0041636, MI0033426), (hsa-miR-10396b-5p, MIMAT0041635, MIMAT0041635, MI0033426), (hsa-miR-10397-3p, MIMAT0041626, MIMAT0041626, MI0033421), (hsa-miR-10397-5p, MIMAT0041625, MIMAT0041625, MI0033421), (hsa-miR-10398-3p, MIMAT0041628, MIMAT0041628, MI0033422), (hsa-miR-10398-5p, MIMAT0041627, MIMAT0041627, MI0033422), (hsa-miR-10399-3p, MIMAT0041630, MIMAT0041630, MI0033423), (hsa-miR-10399-5p, MIMAT0041629, MIMAT0041629, MI0033423), (hsa-miR-103a-1-5p, MIMAT0037306, MIMAT0037306, MI0000109), (hsa-miR-103a-2-5p, MIMAT0009196, MIMAT0009196, MI0000108), (hsa-miR-103a-3p, MIMAT0000101, MIMAT0000101, MI0000108), (hsa-miR-103a-3p, MIMAT0000101_1, MIMAT0000101, MI0000109), (hsa-miR-10400-3p, MIMAT0041632, MIMAT0041632, MI0033424), (hsa-miR-10400-5p, MIMAT0041631, MIMAT0041631, MI0033424), (hsa-miR-10401-3p, MIMAT0041634, MIMAT0041634, MI0033425), (hsa-miR-10401-3p, MIMAT0041634_1, MIMAT0041634, MI0033425), (hsa-miR-10401-3p, MIMAT0041634_2, MIMAT0041634, MI0033425), (hsa-miR-10401-3p, MIMAT0041634_3, MIMAT0041634, MI0033425), (hsa-miR-10401-5p, MIMAT0041633, MIMAT0041633, MI0033425), (hsa-miR-10401-5p, MIMAT0041633_1, MIMAT0041633, MI0033425), (hsa-miR-10401-5p, MIMAT0041633_2, MIMAT0041633, MI0033425), (hsa-miR-10401-5p, MIMAT0041633_3, MIMAT0041633, MI0033425), (hsa-miR-105-3p, MIMAT0004516, MIMAT0004516, MI0000111), (hsa-miR-105-3p, MIMAT0004516_1, MIMAT0004516, MI0000112), (hsa-miR-105-5p, MIMAT0000102, MIMAT0000102, MI0000111), (hsa-miR-105-5p, MIMAT0000102_1, MIMAT0000102, MI0000112), (hsa-miR-10523-5p, MI

In [171]:
# check na (first col in LSCC has lots of na)
# no na in file, but na appear when combining when a miRNA was not included in some files
all_df.columns = all_df.columns.droplevel(['ID','Alias','Derives_from'])
all_df.iloc[:,0].dropna()

Patient_ID
C3L-01663       4.727194
C3L-03678       4.113043
C3L-04014       8.284798
C3N-02283       7.457622
C3N-02288      19.479634
C3N-03441       5.603591
C3L-00923.N     3.308104
C3L-02130.N     2.990037
C3L-02358.N     2.943982
C3L-02552.N     4.337586
C3L-02625.N     4.738640
C3L-02968.N     2.691254
C3N-04124.N     8.639831
Name: hsa-let-7a-2-3p, dtype: float64

In [75]:
ident = 'total'
for can in cancer_names:
    df = pd.read_csv('~/Documents/miRNA/datafreeze/'+can+'_'+ ident +'_miRNA_combined.tsv', delimiter = '\t')
    print(can)
    # check tumor/normal counts
    tumor = [col for col in df.columns if '.N' not in col]
    if ident == 'mature':
        print('Tumor:', len(tumor) - 4)
    else:
        print('Tumor:', len(tumor) - 3)
    normal = [col for col in df.columns if '.N' in col]
    print('Normal:', len(normal))
    #print(df.head())

GBM
Tumor: 99
Normal: 0
HNSCC
Tumor: 111
Normal: 61
LSCC
Tumor: 109
Normal: 97
LUAD
Tumor: 110
Normal: 101
EC
Tumor: 95
Normal: 15
PDA
Tumor: 145
Normal: 38
ccRCC
Tumor: 110
Normal: 75


In [35]:
# total and precursor have same num of tumor and normal
# mature is off by a couple

In [43]:
# Check patient_IDs
new = {}
for can in cancer_names:
    #print(can)
    for i in identifier_list:
        #print(i)
        df = pd.read_csv('~/Documents/miRNA/datafreeze/'+can+'_'+ i +'_miRNA_combined.tsv', delimiter = '\t')
        if i == 'mature':
            new[can+'_'+i] = list(df.columns[4:]) # 'Name', 'ID', 'Alias', 'Derives_from'
        else:
            new[can+'_'+i] = list(df.columns[3:])
        #print(list(df.columns[0:6]))
print('done')

done


In [71]:
#for can in cancer_names:
can = 'PDA'
t = new[can+'_total'] 
m = new[can+'_mature'] 
p = new[can+'_precursor']

l3 = {'total':t,'pre':p, 'mature':m}

for l in l3:
    print(l, len(l3[l]))

total 183
pre 183
mature 183


In [72]:
n = set(t) ^ set(p) 
print('total vs pre:', n, '\n')

n2 = set(t) ^ set(m) 
print('total vs mature:', n2, '\n')

n3 = set(m) ^ set(p) 
print('mature vs pre:', n3)

total vs pre: set() 

total vs mature: set() 

mature vs pre: set()


In [73]:
p

['C3L-00017',
 'C3L-00102',
 'C3L-00189',
 'C3L-00277',
 'C3L-00401',
 'C3L-00589',
 'C3L-00598',
 'C3L-00599',
 'C3L-00622',
 'C3L-00625',
 'C3L-00640',
 'C3L-00819',
 'C3L-00881',
 'C3L-00928',
 'C3L-01031',
 'C3L-01036',
 'C3L-01037',
 'C3L-01051',
 'C3L-01052',
 'C3L-01053',
 'C3L-01054',
 'C3L-01124',
 'C3L-01328',
 'C3L-01453',
 'C3L-01598',
 'C3L-01637',
 'C3L-01662',
 'C3L-01687',
 'C3L-01689',
 'C3L-01703',
 'C3L-01971',
 'C3L-02109',
 'C3L-02112',
 'C3L-02115',
 'C3L-02116',
 'C3L-02118',
 'C3L-02463',
 'C3L-02604',
 'C3L-02606',
 'C3L-02610',
 'C3L-02613',
 'C3L-02701',
 'C3L-02809',
 'C3L-02890',
 'C3L-02897',
 'C3L-02899',
 'C3L-03123',
 'C3L-03129',
 'C3L-03356',
 'C3L-03371',
 'C3L-03388',
 'C3L-03394',
 'C3L-03395',
 'C3L-03628',
 'C3L-03630',
 'C3L-03632',
 'C3L-03635',
 'C3L-03639',
 'C3L-03743',
 'C3L-04027',
 'C3L-04072',
 'C3L-04080',
 'C3L-04473',
 'C3L-04475',
 'C3L-04479',
 'C3L-04495',
 'C3L-04848',
 'C3L-04853',
 'C3N-00198',
 'C3N-00249',
 'C3N-00302',
 'C3N-

In [29]:
dif = list(n2)[0]

if dif in t:
    print('in total')
if dif in m:
    print('in mature')
if dif in p:
    print('in pre')

in total
in pre


In [28]:
dif

'Derives_from'